In [1]:
#http://rmaps.github.io/blog/posts/animated-choropleths/
malnutrition_data_raw <- read.csv('./clean_analysis/UNICEF_child_malnutrition_Sept_2015.csv')

In [2]:
# Distributions would look good as violin plots using ggplot2 - regional aggregrates
summary(malnutrition_data_raw)

       X          ISO_country                              country_name
 Min.   :  0.0   BGD    : 23   BANGLADESH                        : 23  
 1st Qu.:194.2   VEN    : 21   VENEZUELA (BOLIVARIAN REPUBLIC OF): 21  
 Median :388.5   JAM    : 19   JAMAICA                           : 19  
 Mean   :388.5   VNM    : 16   VIET NAM                          : 16  
 3rd Qu.:582.8   CHL    : 15   CHILE                             : 15  
 Max.   :777.0   IDN    : 15   INDONESIA                         : 15  
                 (Other):669   (Other)                           :669  
      Year                  WB_income_region  sample_size      severe_wasting 
 Min.   :1983                       :  1     Min.   :    267   Min.   : 0.00  
 1st Qu.:1995   High income: nonOECD: 66     1st Qu.:   2600   1st Qu.: 0.80  
 Median :2002   High income: OECD   : 29     Median :   4638   Median : 1.80  
 Mean   :2002   Low income          :169     Mean   :  90294   Mean   : 2.41  
 3rd Qu.:2008   Lower middle 

In [3]:
long_dat <- malnutrition_data_raw[,c('Year', 'ISO_country', 'stunting')]

clean_dat <- na.omit(long_dat)

clean_dat$Year <- as.numeric(clean_dat$Year)

# clean_dat$Year <- strptime(clean_dat$Year, "%Y")
# clean_dat$Year <- as.POSIXct(clean_dat$Year)

clean_dat$ISO_country <- as.character(clean_dat$ISO_country)

head(clean_dat)

,Year,ISO_country,stunting
1,1997,AFG,53.2
2,2004,AFG,59.3
3,2013,AFG,40.9
4,1997,ALB,20.4
5,2000,ALB,39.2
6,2005,ALB,27


In [4]:
str(clean_dat)

'data.frame':	744 obs. of  3 variables:
 $ Year       : num  1997 2004 2013 1997 2000 ...
 $ ISO_country: chr  "AFG" "AFG" "AFG" "ALB" ...
 $ stunting   : num  53.2 59.3 40.9 20.4 39.2 27 23.1 16.9 22.9 22.5 ...
 - attr(*, "na.action")=Class 'omit'  Named int [1:34] 58 69 71 72 73 87 116 164 165 166 ...
  .. ..- attr(*, "names")= chr [1:34] "58" "69" "71" "72" ...


In [5]:
#Discretize the data by cutting into sextiles. 
clean_dat_disc <- transform(clean_dat, 
                            fillKey = cut(stunting, quantile(stunting, seq(0,1,1/5)), 
                                                     labels=LETTERS[1:5]))

In [6]:
head(clean_dat_disc)

,Year,ISO_country,stunting,fillKey
1,1997,AFG,53.2,E
2,2004,AFG,59.3,E
3,2013,AFG,40.9,D
4,1997,ALB,20.4,B
5,2000,ALB,39.2,D
6,2005,ALB,27,C


In [7]:
fills = setNames(
    c(RColorBrewer::brewer.pal(5, 'YlOrRd'), 'white'), c(LETTERS[1:5], 'defaultFill')
)

In [17]:
library(plyr)
library(rMaps)

clean_dat_disp <- dlply(na.omit(clean_dat_disc), "Year", function(x){
    y=toJSONArray2(x, json=F)
    names(y)=lapply(y, '[[', 'ISO_country')
    return(y)
})

In [12]:
map2=map$copy()
map2$set(
    bodyattr="ng-app ng-controller='rChartsCtrl'"
)
map2$addAssets(
    jshead="http://cdnjs.cloudflare.com/ajax/libs/angular.js/1.2.1/angular.min.js"
)

In [13]:
map2$setTemplate(chartDiv="
    <div class='container'>
        <input id='slider' type='range' min=1983 max=2015 ng-model='year' width=200>
        <span ng-bind='year'</span>
        <div id='' class='rChart datamaps'></div>
        <script>
            function rChartsCtrl($scope){
                $scope.year=1083;
                $scope.$watch('year', function(newYear)){
                    map.updateChoropleth(chartParams.newData[newYear]);
        })
    }
    </script>"                
)

In [14]:
map2$set(newData=clean_dat_disp)
map2

ERROR: Error in stopifnot(is.list(x), is.list(val)): object 'clean_dat_disp' not found


In [15]:
library(rMaps)
library(rCharts)

ichoropleth <- function(x, data, pal, ncuts, animate, play, map, legend = TRUE, 
                        labels = TRUE, ...){
  d <- Datamaps$new()
  fml = lattice::latticeParseFormula(x, data = data)
  data = transform(data, 
    fillKey = cut(
      fml$left, 
      quantile(fml$left, seq(0, 1, 1/ncuts)),
      ordered_result = TRUE
    )
  )
  fillColors = RColorBrewer::brewer.pal(ncuts, pal)
  d$set(
    scope = map, 
    fills = as.list(setNames(fillColors, levels(data$fillKey))), 
    legend = legend,
    labels = labels,
    ...  
  ) 
if (!is.null(animate)){
    range_ = summary(data[[animate]])
    data = dlply(data, animate, function(x){
      y = toJSONArray2(x, json = F)
      names(y) = lapply(y, '[[', fml$right.name)
      return(y)
    })
    d$set(
      bodyattrs = "ng-app ng-controller='rChartsCtrl'"  
    )
    d$addAssets(
      jshead = "http://cdnjs.cloudflare.com/ajax/libs/angular.js/1.2.1/angular.min.js"
    )
    if (play == T){
      d$setTemplate(chartDiv = sprintf("
        <div class='container'>
         <button ng-click='animateMap()'>Play</button>
         <div id='{{chartId}}' class='rChart datamaps'></div>  
        </div>
        <script>
          function rChartsCtrl($scope, $timeout){
            $scope.year = %s;
              $scope.animateMap = function(){
              if ($scope.year > %s){
                return;
              }
              map{{chartId}}.updateChoropleth(chartParams.newData[$scope.year]);
              $scope.year += 1
              $timeout($scope.animateMap, 1000)
            }
          }
       </script>", range_[1], range_[6])
      )
      
    } else {
      d$setTemplate(chartDiv = sprintf("
        <div class='container'>
          <input id='slider' type='range' min=%s max=%s ng-model='year' width=200>
          <div id='{{chartId}}' class='rChart datamaps'></div>  
        </div>
        <script>
          function rChartsCtrl($scope){
            $scope.year = %s;
            $scope.$watch('year', function(newYear){
              map{{chartId}}.updateChoropleth(chartParams.newData[newYear]);
            })
          }
       </script>", range_[1], range_[6], range_[1])
      )
    }
    d$set(newData = data, data = data[[1]])
    
  } else {
    d$set(data = dlply(data, fml$right.name))
  }
  return(d)
}


Attaching package: ‘rCharts’

The following objects are masked from ‘package:rMaps’:

    Datamaps, Leaflet, makeChoroData, processChoroData, rqMap



In [16]:
ichoropleth(stunting ~ ISO_country, data=clean_dat, pal="Blues", animate="Year", ncuts=5,
            play=T, map='world')